<img src="../../img/logo_amds.png" alt="Logo" style="width: 128px;"/>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2020 Amsterdam UMC - Amsterdam Medical Data Science

# Renal replacement therapy
Shows the mode or types of renal replacement therapy (artificial kidney), if any, a patient received. Includes common renal replacement therapy (RRT) parameters: RRT mode, blood flow rate, substitution fluid, substitution flow rate, net, ultrafiltration rate, filter session time and circuit pressures (arterial, venous, pre-filter, ultrafiltration, transmembrane).

**Note**: Requires creating the [dictionaries](../../dictionaries/create_dictionaries.ipynb) before running this notebook.

## Imports

In [1]:
%matplotlib inline
import amsterdamumcdb
import psycopg2
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl

import io
from IPython.display import display, HTML, Markdown

## Display settings

In [2]:
#matplotlib settings for image size
#needs to be in a different cell from %matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rcParams["figure.dpi"] = 288
plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["font.size"] = 12

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

## Connection settings

In [3]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../../config.ini'):
    config.read('../../config.ini')
else:
    config.read('../../config.SAMPLE.ini')

#Open a connection to the postgres database:
con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
con.set_session(autocommit=True)

cursor = con.cursor()
cursor.execute('SET SCHEMA \'amsterdamumcdb\''); #set search_path to amsterdamumcdb schema

In [4]:
#loads the dictionary for fast lookup
listitems_itemids = pd.read_csv('../../dictionaries/listitems_itemids.csv')
numericitems_itemids = pd.read_csv('../../dictionaries/numericitems_itemids.csv')

## Common renal replacement therapy parameters

### renal replacement mode

In [5]:
listitems_itemids[listitems_itemids['item'].str.contains('MFT', flags=re.IGNORECASE)]

,itemid,item,valueid,value
1478,14846,MFT_Behandeling,1,SCUF
1479,14846,MFT_Behandeling,2,CVVH
1480,14846,MFT_Behandeling,3,HVCVVH
1481,14846,MFT_Behandeling,4,CVVHDF
1482,14846,MFT_Behandeling,6,MPS


### substitution fluids used
Also known as replacement fluids.

In [6]:
listitems_itemids[listitems_itemids['item'].str.contains(r'subst', flags=re.IGNORECASE)]

,itemid,item,valueid,value
70,7672,Substitutie vloeistof,1,BH504
71,7672,Substitutie vloeistof,2,Citraat
72,7672,Substitutie vloeistof,4,HF32bic
73,7672,Substitutie vloeistof,6,BH504/HF32bic
4427,20548,CVVH behandel afspraken Substitutie PostDilutie,1,Bicarbonaat


### blood flow rate

In [7]:
#possible renal replacement parameters from, numericitems
numericitems_itemids[numericitems_itemids['item'].str.contains('(?<!ECMO.-.)bloed.*flow', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
587,10736,Bloed-flow,24,ml/min
951,12444,Bloedflow ingesteld,24,ml/min
966,12460,Bloedflow,24,ml/min
1176,14850,MFT_Bloedflow (ingesteld),24,ml/min
1637,20538,CVVH behandel afspraken Bloedflow,24,ml/min


### substitution flow rate (prescribed)
Substitution flow rate,  predilution (=pre-filter substitution) flow rate, postdilution (=post-filter substitution) flow rate. Expressed as 'ml/h' or just 'ml'.

In [8]:
numericitems_itemids[numericitems_itemids['item'].str.contains('(?:substituaat|substitutie|dilutie)(?!volume_totaal)(?!volume_huidig)', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
586,10735,Substitutie-flow,46,ml/uur
958,12451,Substitutie vloeistof volume ingesteld,6,ml
959,12452,Substitutieflow ingesteld,46,ml/uur
967,12461,Substitutieflow,46,ml/uur
1174,14848,MFT_Substituaat (ingesteld),46,ml/uur
1640,20544,CVVH behandel afspraken PostDilutie Volume,6,ml
1641,20547,CVVH behandel afspraken PreDilutie Volume,6,ml
1660,20710,MFT_Predilutieflow (ingesteld),46,ml/uur
1661,20716,MFT_Postdilutieflow (ingesteld),46,ml/uur


### substitution volume (delivered)

In [9]:
numericitems_itemids[numericitems_itemids['item'].str.contains('(?:substituaat|substitutie|dilutie).*(?:_huidig|_totaal)', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
1617,20076,MFT_Substitutievolume_totaal,8,l
1618,20077,MFT_Substitutievolume_huidig,8,l
1656,20706,MFT_Predilutievolume_huidig,8,l
1657,20707,MFT_Predilutievolume_totaal,8,l
1658,20708,MFT_Postdilutievolume_huidig,8,l
1659,20709,MFT_Postdilutievolume_totaal,8,l


### Ultrafiltration rate (prescribed)

In [10]:
numericitems_itemids[numericitems_itemids['item'].str.contains('(?:UF\s|ultrafiltratie|filtraatvolume)(?!_totaal)(?!_huidig)', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
1175,14849,MFT_Ultrafiltratie (ingesteld),46,ml/uur
1177,14851,MFT_UF Totaal (ingesteld),6,ml


### Ultrafiltration achieved

In [11]:
numericitems_itemids[numericitems_itemids['item'].str.contains(r'(?:UF\s|ultrafiltratie|filtraatvolume).*(?:_huidig|_totaal)', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
1619,20078,MFT_Filtraatvolume_huidig,8,l
1620,20079,MFT_Filtraatvolume_totaal,8,l


### Filter session time

In [12]:
numericitems_itemids[numericitems_itemids['item'].str.contains('MFT_FilterLooptijd', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
1630,20340,MFT_FilterLooptijd,5,uur


### Pre-filter pressure
Also known as Pre-blood entry (PBE) pressure

In [13]:
numericitems_itemids[numericitems_itemids['item'].str.contains(r'Pre-filter|PBE', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
123,7667,Pre Filter PBE,173,mmHg
953,12446,PBE max ingesteld,173,mmHg
1167,14839,MFT_Pre-filter druk,173,mmHg


### Arterial pressure
Also known as access pressure, pressure at circuit entry (negative pressure)

In [14]:
numericitems_itemids[numericitems_itemids['item'].str.contains('art.*druk', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
122,7666,Arteriele druk,173,mmHg
1163,14835,MFT_Arteriele druk,173,mmHg


### Venous pressure
Also known as return pressure, pressure at circuit exit

In [15]:
numericitems_itemids[numericitems_itemids['item'].str.contains(r'veneu.*druk', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
124,7668,Veneuze druk braun,173,mmHg
1165,14837,MFT_Veneuze druk,173,mmHg


### Ultrafiltration pressure
Also known as effluent pressure (or PD2)

In [16]:
numericitems_itemids[numericitems_itemids['item'].str.contains('(?<!pre-)filt.*druk', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
125,7671,Negatieve filterdruk,173,mmHg
1164,14836,MFT_Filtraat druk,173,mmHg


### Transmembrane pressure

In [17]:
numericitems_itemids[numericitems_itemids['item'].str.contains('tmp', flags=re.IGNORECASE)]

,itemid,item,unitid,unit
960,12453,TMP max ingesteld,173,mmHg
965,12459,TMP,173,mmHg
1166,14838,MFT_TMP,173,mmHg
1168,14840,MFT_TMP bovengrens,173,mmHg
1173,14845,MFT_TMP ondergrens,173,mmHg


## Renal replacement therapy: summary per admission

In [18]:
sql_renal_replacement = """
WITH rrt_bloodflow AS (
    SELECT 
        n.admissionid,
    CASE
        WHEN COUNT(*) > 0 THEN TRUE
        ELSE FALSE
    END AS renal_replacement_bool,
    MAX(n.value) AS renal_replacement_max_blood_flow
    FROM numericitems n
    WHERE n.itemid IN (
            10736, --Bloed-flow
            12460, --Bloedflow
            14850 --MFT_Bloedflow (ingesteld): Fresenius multiFiltrate blood flow
        )
        AND n.value > 0
    GROUP BY n.admissionid
), rrt_mode AS (    
    SELECT 
    admissionid,
    STRING_AGG(DISTINCT value, '; ') AS renal_replacement_modes
    FROM listitems l 
    WHERE l.itemid IN (
            14846 -- MFT_Behandeling: Fresenius multiFiltrate mode
        )
    GROUP BY l.admissionid
)
SELECT bf.*, m.renal_replacement_modes
FROM rrt_bloodflow bf
LEFT JOIN rrt_mode m ON
    bf.admissionid = m.admissionid
"""            
renal_replacement = pd.read_sql(sql_renal_replacement,con)
renal_replacement.head()

,admissionid,renal_replacement_bool,renal_replacement_max_blood_flow,renal_replacement_modes
0,37,True,180.0,HVCVVH
1,47,True,180.0,HVCVVH
2,54,True,180.0,CVVH
3,194,True,250.0,CVVH
4,197,True,220.0,CVVH
